In [231]:
import pandas as pd
import re # re.search etc. regular expressions

Create the Reference Table:
Create a DataFrame that defines your categories, subcategories, and corresponding descriptions.

In [232]:
# Reference table for categories and subcategories
# ???? CENTER FOR INTUITIVE P WARREN OH
# ???? BEACHWOOD PLACE, OH-HC BEACHWOOD OH
categories_data = {
    # INCOME TAXES FEES ETC EXCLUDE PAYMENTS
    'Income': {
        'Anita Income': ['ZELLE DEP ANITA'],
        'Fidelity Transfer': ['FID '],
        'KeyBank Cash-Back': ['KEY REWARDS', 'GIFT FROM KEY BANK']
    },
    'Taxes': {
        'Taxes': ['TAXREFUND', ' IRS ', 'TAX REF', 'RITA']
    },
    'Fees': {
        'Transaction Fees': ['TRANSACTION FEE']
    },
    'EXCLUDE': {
        'Visa Payment': ['INTERNET TRF TO CCA'],
        'Visa Payment Received': ['PAYMENT ']
    },
    # PRESUMABLY CHECKING ACCT
    'Housing': {
        'Mortgage': ['WFHM'],
        'Bills n Utilities': ['VERIZON', 'VZWRLSS', 'DOMINION', 'FIRST ENERGY', 'NORTHEAST OHIO', 'CLEVELAND HEIGHTS', 'ENBRIDGE GAS', 'ATT ', 'NEORSD', 'CWD']
    },
    'Insurance': {
        'Medical Insurance': ['MEDICARE', 'VSP', 'UNITEDHEALTHCARE', 'ROCKWELL', 'AARP HEALTH', 'DELTA DENTAL'],
        'Car Insurance': ['LIBERTY MUTUAL']
    },
    'Education': {
        'College Tuition': ['SMARTPAYCIA', 'CASHNET', 'CAMPUS CIA', 'BURREN', 'COLLEGE'],
        'Art Supplies': ['UTRECHT', ' ART ']
    },
    'Professional': {
        'Professional Fees': ['LICENSURE', 'LICENSE'],
        'Liability Insurance': ['CPH LIABILITY']
    },
    # PRESUMABLY CREDIT CARD
    'Transportation': {
        'Gas': ['SUNOCO', 'BP', 'SHELL', 'MARATHON', 'CIRCLE K', 'SHEETZ', 'GAS'],
        'Car Maintenance': ['REPAIR', 'AUTO', 'BODY', 'QUALITY AUTO'],
        'Car Registration' : ['BUREAU MOTOR VE']
    },
    'Food': {
        'Groceries': ['GROCERY', 'HEINEN', 'DAVE', 'WHOLE', 'SODA', 'TRADE'],
        'Dining Out': ['TAVERN', 'TOMMYS', 'CAFE', 'WASABI', 'PACIFIC', 'ANATOLIA', 'BATUQUI', 'PHO', 'LAKE HOUSE', 'DEWEY',\
                        'MAROTTA ', 'BANANA', 'BANGKOK', 'HIBACHI', 'BRASSICA', 'RESTAUR', 'BUFFALO', 'COZUMEL', 'FIRST WATCH',\
                            'PARADISE BIRYANI', 'CARIBOU COFFEE', 'STONE OVEN', 'SEOUL GARDEN ', 'YOURS TRULY', 'LOCKKEEPERS' ],
        'Fast Food': ['LEFTY', 'SHAKE SHACK', 'SHAKESHACK', 'WENDY', 'BUDDA', 'CILANTRO', 'PANERA', 'CHIPOTLE', 'BIBIBOP', 'ROGERS', 'PIADA',\
                      'ZINA', 'SUBSHOPPE', 'NATURES OASIS', 'LOTUS EXPRESS', 'BRUEGGERS '],
        'World Food': ['KRAKOW', 'NIPA HUT', 'YELESEYEVSKY'],
        'Why DOORDASH?': ['DOORDASH']
    },
    'Health & Beauty': {
        'Aikido n Yoga': ['CHECK', ' YOGA'], 
        'Beauty n Supplies': ['LADIES', 'BATH', 'SALLY BEAUTY', 'AVEDA', 'LUSH BEACHWOOD', 'AIKIKAI', 'AIKIDO', 'ATMA', 'PADDLE'],
        'Sound Bath': ['PAYPAL INST'], # ?????
         # CLEVELAND KIDNEY & HYP - tomek's kidney USG ?
        'Medical and Dental': ['PEDIATRICS', 'CLEVELAND CLINIC', 'METROHEALTH', 'WESTERN RESERVE PERIO', 'HILLCREST ', 'CLEVELAND KIDNEY '],
        'Pharmacy': ['CVS', 'WALGREENS']
    },
    'Home & Garden': {
        'House Maintenance': ['HOME DEPOT'],
        'Furnishing': ['WORLD', 'REFURNISHING', 'KOALA', 'WAYFAIR'],
        'Garden': ['BREMEC', 'LANDSCAPE', 'STUMP', 'NATURE CENTER']
    },
    'Subscriptions': {
        'Subscription': ['SPOTIFY', 'APPLE', 'NETFLIX', 'AUDIBLE', 'PEACOCK', 'WALL', 'BITDEFENDER', 'MICROSOFT', 'HULU', 'NYTIMES', 'IDEASTREAM', 'WSJ']
    },
    'Shopping': {
        'Amazon': ['AMAZON', 'AMZN'],
        'Department Store': ['TARGET', 'MACY'],
        # BEACHWOOD PLACE, OH-HC BEACHWOOD OH =? Ann Taylor store in beachwood place
        'Clothes': ['REI', 'NORDSTROM', 'DICK', 'DSW', 'AVALON', 'MARSHALLS', 'ANN TAYLOR', 'AMERICAN EAGLE', 'FOOTWEAR', 'H&M ', 'OLD NAVY '],
        'Kindle n Books': ['KINDLE', 'AUDIOTEKA', 'LOGANBERRY', 'MAC\'S', 'EMPIK'],
        # SERIF.COMBILL MINNETONKA MN = AFFINITY GRAPHICS SOFTWARE
        # SIMON HAYNES T AIAT PAYPAL = SpaceJock Writing software
        'Software n Accessories': ['SIMON HAYNES', 'ALISTORE', 'GOOGLE', 'FLIXEASY', 'CLIP STUDIO', 'ALIEXPRESS', 'SERIF.COMBILL MINNETONKA MN'],
        # LOTUS FLOWER LLC = flowers to Poland
        'Gifts': ['FIDDLEHEAD', 'PASSPORT', 'DIAMONDS FLOWERS', 'BUNDT', 'ALL CITY CANDY', 'LOTUS FLOWER LLC', 'PINKBLUSHMATERNIT',\
                  'LITTLE ROOM ', 'CRAFT COLLECTIVE'],
        # THE CHILDRENS PLACE 42 BEACHWOOD OH 
        # THECHILDRENSPLACE.COM SECAUCUS NJ
        'Kids Toys': ['PLAYMATTERS', 'DISNEYSTORE', 'CHILDRENS PLACE', 'THECHILDRENSPLACE.COM' ],
        'Cycling n Paddling' : ['BIKES', 'IROCKER']
    },
    'Pets': {
        'Cat Food n Supplies': ['PET', 'CHEWY', 'HOLLYWOOD FEED', 'JACKSON GALAXY'],
        'Cat Health': ['VETERINARY']
    },
    # ENTERTAINMENT n GIFTS
            # MOVIES THEATER COMPUTER GAMES BIRTHDAY CAKES BDAY DINNERS ETC
            # NOTE: CLEVELAND INSTITUTE OF CLEVELAND ==== CIA CINEMATEQUE
            # MUSEUM ===== includes THE CHILDRENS MUSEUM O CLEVELAND OH 
            # GREATER CLEVELAND AQUA ==== CLEVELAND AQUARIUM
            # MANGO MANGO DESSE CLEVELAND ===== mango mango dessert Sof's birthday with Felix
            # DIAMONDS FLOWERS ========= flower shop Coventry
    'Entertainment': {
        # METROPARKS FARMPA KIRTLAND - Farm Park Kirtland w Kiddos
        'Fun Out': ['CLEVELAND MUSEUM OF AR', 'GREATER CLEVELAND AQUA', 'METROPARKS FARMPA KIRTLAND', 'MITCHELL',\
                                      'SWEET FIX', 'MANGO MANGO DESSE',\
                                          'ON THE RISE', 'RISING STAR COFFEE', 'STARBUCKS', 'MICHAELS', 'UPTOWN MART', 'ELLIE-MAYS', '6 FLAVORS INDIAN', 'LUXE KITCHEN'],
        'Movies n Theater': ['VUDU', 'FANDANGO', 'DOBAMA','THEAT', 'CLEVELAND PUBLIC', 'CLEVELAND INSTITUTE OF CLEVELAND',\
                                   'MOVIE', 'PRIME VIDEO', 'BORDERLIGHT', 'CINEMA'],
        'Music n Games': ['STEAMGAMES', 'BANDCAMP'],
        # Cleveland Museum Cleveland OH - Cleveland Museum of Art membership
        'Memberships': ['Cleveland Museum Cleveland OH']
    },
    # PY *PCRF LOS ANGELES CA - Palestine children relief fund ??? 
    'Miscellaneous': {
        'Political Donations': ['ACTBLUE'],
        'ATM Wthdrw n Dpsit' : ['ATM '],
        'Vending Machines' : ['VENDING ', 'PEPSIVEN'],
        'Parcels' : ['USPS ']
    },
    # EXCEPTIONS - PROCESS LAST
    'Travel': {
        # HNDISCOVER ST728 CLEVELAND OH = Hopkins airport  
        # Espresso House 7187 Kastrup DNK = Copenhagen Airport
        'Air Travel': ['LOT ', 'AMERICAN', 'EXPEDIA', 'SAS ', 'EAT AND GO JAMAICA', ' Kastrup', 'HUDSONNEWS ', 'HUDSON',\
                       'HNDISCOVER', 'JFK ', 'MIDTOWN BISTRO', 'TST* ', 'CURRITO '],
        'Visiting Grandma Ela': ['PLUSKI', 'OLSZTYN', 'WARSZAWA', re.compile(r' POL$'), re.compile(r' CHICAGO IL$')],
        'Vacation SC': ['FOLLY', 'VIATORTRIPADVISOR', 'VACASA', 'VRBO', re.compile(r' SC$'), re.compile(r' NC$'), re.compile(r' WV$')],
        'Visiting Wanda': [re.compile(r' VA$'), re.compile(r' GA$'), re.compile(r' MD$')],
        'Visiting Eva': ['PITTSBURGH', 'TURNPIKE', re.compile(r' PA$')]
    },
}

Verify Integrity of categories definition

In [233]:
def find_duplicates(data):
    subcategory_seen = set()
    keyword_seen = set()
    duplicate_subcategories = []
    duplicate_keywords = []

    for category, subcategories in data.items():
        for subcategory, keywords in subcategories.items():
            # Check for duplicate subcategories
            if subcategory in subcategory_seen:
                duplicate_subcategories.append(subcategory)
            else:
                subcategory_seen.add(subcategory)
            
            # Check for duplicate keywords
            for keyword in keywords:
                if isinstance(keyword, re.Pattern):
                    keyword = keyword.pattern  # Convert regex to string for comparison
                
                if keyword in keyword_seen:
                    duplicate_keywords.append(keyword)
                else:
                    keyword_seen.add(keyword)
    
    return duplicate_subcategories, duplicate_keywords

In [234]:
def find_duplicates_and_substrings(data):
    subcategory_seen = set()
    keyword_seen = set()
    duplicate_subcategories = []
    duplicate_keywords = []
    substring_subcategories = []
    substring_keywords = []

    for category, subcategories in data.items():
        for subcategory, keywords in subcategories.items():
            # Check for duplicate subcategories
            if subcategory in subcategory_seen:
                duplicate_subcategories.append(subcategory)
            else:
                subcategory_seen.add(subcategory)
            
            # Check for subcategory substrings
            for seen_sub in subcategory_seen:
                if seen_sub != subcategory and (subcategory in seen_sub or seen_sub in subcategory):
                    substring_subcategories.append((subcategory, seen_sub))
            
            # Check for duplicate keywords
            for keyword in keywords:
                if isinstance(keyword, re.Pattern):
                    keyword_str = keyword.pattern  # Convert regex to string for comparison
                else:
                    keyword_str = keyword
                
                if keyword_str in keyword_seen:
                    duplicate_keywords.append(keyword_str)
                else:
                    keyword_seen.add(keyword_str)

                # Check for keyword substrings
                for seen_kw in keyword_seen:
                    if seen_kw != keyword_str and (keyword_str in seen_kw or seen_kw in keyword_str):
                        substring_keywords.append((keyword_str, seen_kw))
    
    return duplicate_subcategories, duplicate_keywords, substring_subcategories, substring_keywords

In [ ]:
# Find duplicates and substrings in categories_data
duplicate_subcategories, duplicate_keywords, substring_subcategories, substring_keywords = find_duplicates_and_substrings(categories_data)

# Print the results
print("Duplicate Subcategories:")
print(duplicate_subcategories if duplicate_subcategories else "None")

print("\nDuplicate Keywords:")
print(duplicate_keywords if duplicate_keywords else "None")

print("\nSubstring Subcategories:")
print(substring_subcategories if substring_subcategories else "None")

print("\nSubstring Keywords:")
print(substring_keywords if substring_keywords else "None")

In [ ]:
# Find duplicates in categories_data
duplicate_subcategories, duplicate_keywords = find_duplicates(categories_data)

# Print the results
print("Duplicate Subcategories:")
print(duplicate_subcategories if duplicate_subcategories else "None")

print("\nDuplicate Keywords:")
print(duplicate_keywords if duplicate_keywords else "None")

In [ ]:

# Convert the hierarchical dictionary into a DataFrame for easier processing
categories_list = []
for category, subcategories in categories_data.items():
    for subcategory, keywords in subcategories.items():
        categories_list.append({'Category': category, 'Subcategory': subcategory, 'DescriptionKeywords': keywords})

categories_df = pd.DataFrame(categories_list)

print(categories_list)
print(categories_df)

Function to Categorize Transactions:
Create a function that uses the reference table to categorize transactions based on the description.

In [ ]:
# Using iterrows() to iterate over rows
for index, row in categories_df.iterrows():
    print(f"Index: {index}")
    print(f"Row: {row}")

In [239]:
def categorize_transaction(description, categories_df):
    description_upper = description.upper() 
    for index, row in categories_df.iterrows():
        for keyword in row['DescriptionKeywords']:
            if isinstance(keyword, str) and keyword in description_upper:
                return row['Category'], row['Subcategory']
            elif isinstance(keyword, re.Pattern) and keyword.search(description_upper):
                return row['Category'], row['Subcategory']
    return 'Uncategorized', 'Uncategorized'

In [240]:
def load_and_append(file_path, current_df):
    """
    Load an Excell file and append its content to the current DataFrame.

    Parameters:
    - file_path: str, the path to the new file
    - current_df: pd.DataFrame, the current DataFrame

    Returns:
    - pd.DataFrame, the updated DataFrame with the new data appended
    """
    # Load the new DataFrame from the Excell file
    new_df = pd.read_excel(file_path)

    # Append the new DataFrame to the current DataFrame
    updated_df = pd.concat([current_df, new_df], ignore_index=True)

    return updated_df

In [241]:
def load_expenses(year_month):
    # Load the CSV files with credit card and checking card records
    creditcard_filename = year_month + '-A&T-CCard.xlsx'
    checking_filename = year_month + '-A&T-CheckingAcct.xlsx'

    df_credit = pd.read_excel(creditcard_filename)
    df_checking = pd.read_excel(checking_filename)
    print(df_credit.head())
    print(df_checking.head())

    # Check the first few rows to understand the structure of your data.
    df = pd.concat([df_checking, df_credit], ignore_index=True)
    df.head()
    return df

In [ ]:
months_to_process = ['2024-06','2024-07', '2024-08', '2024-09','2024-10','2024-11','2024-12','2025-01']

df  = pd.DataFrame()

for month in months_to_process : 
    df_month = load_expenses(month)
    df = pd.concat([df, df_month], ignore_index=True)

# Extract the year from the first month in the list
year = months_to_process[0][:4]

# Extract the months from each entry in the list
months = [month[5:] for month in months_to_process]

# Concatenate the year and the months together in the desired format
# Join the list of months into a single string
months_string = "-".join(months)

# Create the final result string
result_string = f"{year}-{months_string}"
output_datafile_summary = result_string + '-A&T-SUMMARY.csv'
categorized_expenses_filename = result_string + '-A&T-CATEGORIZED.csv'

print(output_datafile_summary)
print(categorized_expenses_filename)

In [243]:
# df = load_and_append('2024-10-A&T-CCard.xlsx', df)
# df = load_and_append('2024-10-A&T-CheckingAcct.xlsx', df)
# df = load_and_append('2024-11-A&T-CCard.xlsx', df)
# df = load_and_append('2024-11-A&T-CheckingAcct.xlsx', df)
# df = load_and_append('2024-12-A&T-CCard.xlsx', df)
# df = load_and_append('2024-12-A&T-CheckingAcct.xlsx', df)

Categorize the transactions:

Add a new column for categories. You can create a function to categorize transactions based on the description or other columns.

In [ ]:

# Apply the function to categorize transactions
df[['Category', 'Subcategory']] = df['Description'].apply(lambda x: pd.Series(categorize_transaction(x, categories_df)))

# Filter out rows with 'EXCLUDE' category
filtered_df = df[df['Category'] != 'EXCLUDE']

sorted_df = filtered_df.sort_values(by=['Category', 'Subcategory'])
# Print the categorized DataFrame
print(sorted_df)

sorted_df.to_csv(categorized_expenses_filename)

Convert the date column to datetime:

Ensure the date column is in datetime format.

In [ ]:
# Convert the date column to datetime:
# Ensure the date column is in datetime format.

sorted_df['Date'] = pd.to_datetime(sorted_df['Date'])

#Group by month:
# Extract the month and year from the date column and group the data by these values.

sorted_df['YearMonth'] = sorted_df['Date'].dt.to_period('M')

# Summarize expenses by category for each month
monthly_expenses = sorted_df.groupby(['YearMonth', 'Category'])['Amount'].sum().unstack().fillna(0)

# Transpose the DataFrame
monthly_expenses_transposed = monthly_expenses.T

# Reset index to make 'Category' a column
monthly_expenses_transposed = monthly_expenses_transposed.reset_index()

# Rename columns for clarity
monthly_expenses_transposed.columns.name = None

# Print the final DataFrame
print(monthly_expenses_transposed)
monthly_expenses_transposed.to_csv(output_datafile_summary, index=False)

Analyze and visualize the data:

You can now analyze the monthly expenses by category and create visualizations.

In [ ]:
import matplotlib.pyplot as plt

# Reorder expense categories for display
display_order = ['Income',\
                'Housing',\
                'Insurance',\
                'Food',\
                'Subscriptions',\
                'Transportation',\
                'Health & Beauty',\
                'Shopping',\
                'Home & Garden',\
                'Pets',\
                'Entertainment',\
                'Professional',\
                'Taxes',\
                'Education',\
                'Travel',\
                'Fees',\
                'Uncategorized'
                    ]
# Reorder the columns in the DataFrame according to the desired order
monthly_expenses = monthly_expenses[display_order]


# Define your custom colors for each category
# Get the list of unique categories in your data
categories = monthly_expenses.columns
# Map the colors to the categories
colors = [category_colors[cat] for cat in categories]
category_colors = {
    'Food': '#1f77b4',  # Blue
    'Shopping': '#FF6666', # Light Red
    'Housing': '#2ca02c',  # Green
    'Income' : '#D3D3D3', # Light Gray 
    'Travel' : '#8B0000', # Dark Red
    'Education' : '#800080', # Purple
    'Entertainment' : '#FF00FF', # Magenta
    'Fees'  : '#FF0000', # Red
    'Health & Beauty' : '#FFC0CB', # Pink
    'Home & Garden' : '#FFA500', # Orange
    'Insurance' : '#FFFF00', # Yellow
    'EXCLUDE' : '#FF6666', # Light Red
    'Pets' : '#8A2BE2', # Violet
    'Miscellaneous' : '#708090', # Slate Gray
    'Subscriptions' : '#E6E6FA', # Lavender
    'Taxes' : '#696969', # Dim Gray
    'Transportation' : '#708090', # Slate Gray
    'Uncategorized' : '#000000', # Black
    'Professional' : '#FF8C00', # Dark Orange

    # Add more categories and colors as needed
    # '#FFFF00', # Yellow
    # '#FF0000', # Red
    # '#800080', # Purple
    # '#8A2BE2', # Violet
    # '#FFA500', # Orange
    # '#FFC0CB', # Pink
    # '#FF00FF', # Magenta
    # '#D8BFD8', # Light Purple
    # '#E6E6FA', # Lavender
    # '#FF6666', # Light Red
    # '#FFFFE0', # Light Yellow
    # '#FF8C00', # Dark Orange
    # '#8B0000', # Dark Red
    # '#FF1493', # Deep Pink
    # '#D3D3D3', # Light Gray 
    # '#708090', # Slate Gray
    # '#696969', # Dim Gray
    # '#000000', # Black
}

# Plot monthly expenses by category
monthly_expenses.plot(kind='bar', stacked=True, figsize=(10, 6), color=colors, width=1)
plt.title('Monthly Expenses by Category')
plt.xlabel('Month')
plt.ylabel('Amount ($)')

# Move the legend to the best location
# plt.legend(loc='best')
# Move the legend to the outside right of the plot
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.show()